# Melbourne Housing
Analysis by Grace Wamaitha


## Introduction
The Melbourne Housing Company (MHC) is a real estate agency specializing in the sale of residential properties across Melbourne and its surrounding regions. The company works closely with homeowners, property developers, and prospective buyers to facilitate smooth and transparent property transactions. As part of its operations, MHC collects detailed information on each property it manages-from physical attributes such as the number of rooms, bedrooms, bathrooms, car spaces, land size and building area as well as contextual factors including surburb, region, postcode and distance from key locations(CBD-Central Business District) and temporal information such as sale date along with metadata such as the type of house and the sales person involved.

## Data Dictionary
* Suburb -suburb in which the property is located
* Rooms - Number of rooms in the property
* Type - Type of the property like
    * h - house, cottage,villa,semi,terrace
    * t - townhouse
    * u - unit,duplex
* SellerG - Name of the real estate agent who sold the property
* Date - Date on which the property was sold
* Distance - Distance of the property from CBD in kilometres.
* Postcode - postcode of the area
* Bedroom - number of bedrooms in the property
* Bathroom - number of bathrooms in the property
* Car - Number of car parking space in the property
* Landsize - Size of the land in square metres
* Building Area - size of the building in square metres
* Yearbuilt - Year in which the building was built
* Regionname - Name of the region in which the property is located
* Propertycount - Number of properties that are present in the suburb
* Price - price in AUD at which the property was sold

## Problem statement
The Melbourne Housing Company seeks to **improve the accuracy and consistency of its property valuation process**. Currently, it lacks a reliable analytical framework to translate property attributes into precise price estimates. This limitation affects the agency's ability to guide clients confidently, set competitive listing prices, and **anticipate market behaviour**. To address this challenge, the company aims to leverage its historical sales data to uncover the key *factors that influence housing prices* and develop a **predictive** model capable of approximating the expected selling price of a property based on its physical and contextual characteristics; this will subsequently strenghten decision-making, support its sales team with data-driven insights, enhance customer trust, and maintain competitivenss in Melbourne dynamic housing market.


## Analysis approach
* **Pricing and Valuation**
    1. Which physical features (rooms, bedrooms, bathrooms, car spaces, land size, building area) have the strongest influence on house prices?
    2. How does distance from the CBD affect price?
    3. Can we predict the price of a property accurately using the available features?
    4. How much more (or less) do properties in certain suburbs or regions sell for compared to others?
    5. How does the year a house was built influence its selling price?
* **Market Trends and Demand**
    1. What are the seasonal patterns in property sales?(Do prices rise in certain months?)
    2. Are certain suburbs experiencing increasing or decreasing prices over time?
    3. Which property types(house, unit, townhouse) are performing best in the current market?
    4. How does property count in a suburb relate to the average price?
* **Agent and Operational Performance**
    1. Whhich salespersons consistently achieve higher sales prices or faster sales?
    2. Is there a difference in property price performance between different sales agents?
    3. Does property type or location influence which salesperson should be assigned to a listing?
* **Customer and Location insights**
    1. What characteristics describe the most expensive vs most affordable suburbs?
    2. How do surburbs differ in terms of average land size, building area, or number of rooms?
    3. Which regions show the fastest market growth or highest demand?
* **Property Characteristic and Optimization**
    1. What combination of features yeilds the best return on investment for sellers?
    2. Is adding an extra bedroom, bathroom, or parking space associated with a sustantial Increase in price?
    3. How do older houses compare to newer ones in terms of price per square meter?
* **Data Quality and Strategic Insights**
    1. Are there inconsistencies in the collected data that may hinder pricing accuracy?
    2. Which features should be prioritized when collecting new data to improve future predictions?

### Importing libraries and loading data

In [4]:
# Libraries for data manipulation
import pandas as pd
import numpy as np
# Libraries for visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Restrict float values to a precision on 3 decimal places
pd.set_option("display.float_format",'{:.3f}'.format)

In [7]:
# Loading Melbourne Housing dataset
df = pd.read_csv("Melbourne_Housing.csv")
df.head()

,Suburb,Rooms,Type,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount,Price
0,Airport West,3,t,Nelson,03-09-2016,13.500,3042.000,3.000,2.000,1.000,303.000,225,2016.000,Western Metropolitan,3464,840000
1,Albert Park,2,h,hockingstuart,03-09-2016,3.300,3206.000,2.000,1.000,0.000,120.000,82,1900.000,Southern Metropolitan,3280,1275000
2,Albert Park,2,h,Thomson,03-09-2016,3.300,3206.000,2.000,1.000,0.000,159.000,inf,NaN,Southern Metropolitan,3280,1455000
3,Alphington,4,h,Brace,03-09-2016,6.400,3078.000,3.000,2.000,4.000,853.000,263,1930.000,Northern Metropolitan,2211,2000000
4,Alphington,3,h,Jellis,03-09-2016,6.400,3078.000,3.000,2.000,2.000,208.000,inf,2013.000,Northern Metropolitan,2211,1110000


### Data Profiling

In [8]:
# Check the number of records and variables present in the data
df.shape

(27114, 16)

There are 27114 records and 16 variables.

In [9]:
# Check the datatypes of the variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27114 entries, 0 to 27113
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27114 non-null  object 
 1   Rooms          27114 non-null  int64  
 2   Type           27114 non-null  object 
 3   SellerG        27114 non-null  object 
 4   Date           27114 non-null  object 
 5   Distance       27113 non-null  float64
 6   Postcode       27113 non-null  float64
 7   Bedroom        20678 non-null  float64
 8   Bathroom       20672 non-null  float64
 9   Car            20297 non-null  float64
 10  Landsize       17873 non-null  float64
 11  BuildingArea   10543 non-null  object 
 12  YearBuilt      11985 non-null  float64
 13  Regionname     27114 non-null  object 
 14  Propertycount  27114 non-null  int64  
 15  Price          27114 non-null  int64  
dtypes: float64(7), int64(3), object(6)
memory usage: 3.3+ MB


* Notice that 'Date' variable has been represented as a categorical variable. We will fix the datatype to the correct 'datetime' type.
* The postcode has been given as a float variable. It serves as a location describer, thus categorical. We will also fix its datatype to 'object'.
* The number of bedrooms, bathrooms and car parking spaces are discrete variables. We will also fix their datatypes to int.
* Building area has been read as an object variable, but it should be numerical. We will look at what could be causing this behaviour and fix it.
* Year built should be and interger instead of floating value.
* Notice that some variables have missing values (less than 27114 entries).

#### Fixing the data types of variables

**Fixing the datatype of 'Date' variable from object to datetime**

In [12]:
# Let's confirm the format of the dates
df["Date"].unique()

array(['03-09-2016', '03-12-2016', '04-02-2016', '04-06-2016',
       '06-08-2016', '07-05-2016', '07-11-2016', '08-10-2016',
       '10-09-2016', '10-12-2016', '12-06-2016', '12-11-2016',
       '01-07-2017', '03-06-2017', '03-09-2017', '04-03-2017',
       '04-11-2017', '06-05-2017', '07-10-2017', '08-04-2017',
       '08-07-2017', '09-09-2017', '09-12-2017', '11-02-2017',
       '11-03-2017', '11-11-2017', '12-08-2017', '03-02-2018',
       '03-03-2018', '06-01-2018', '10-02-2018', '10-03-2018',
       '13-05-2017', '13-08-2016', '14-05-2016', '14-10-2017',
       '15-07-2017', '15-10-2016', '16-04-2016', '16-07-2016',
       '16-09-2017', '17-02-2018', '17-03-2018', '17-06-2017',
       '17-09-2016', '18-03-2017', '18-06-2016', '18-11-2017',
       '19-08-2017', '19-11-2016', '20-01-2018', '20-05-2017',
       '21-10-2017', '22-04-2017', '22-05-2016', '22-07-2017',
       '22-08-2016', '23-04-2016', '23-09-2017', '24-02-2018',
       '24-06-2017', '24-09-2016', '25-02-2017', '25-11

We can see that all the dates are consistent. Now let's convert the datatype.

In [19]:
# Converting 'Date' from object type to datetime
df["Date"] = pd.to_datetime(df["Date"],dayfirst=True)
# Confirming the changes
df["Date"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 27114 entries, 0 to 27113
Series name: Date
Non-Null Count  Dtype         
--------------  -----         
27114 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 212.0 KB


We have successfully  changed the datatype of 'Date' variable to 'datetime'.

**Fixing the datatype of 'Postcode' variable from float to object**

In [29]:
# Check the format in which the values were recorded
df.loc[10:15,"Postcode"]

10   3032.000
11   3147.000
12   3147.000
13   3147.000
14   3034.000
15   3034.000
Name: Postcode, dtype: float64

Indeed, the values were recorded with decimal places.

In [35]:
# Convert postcode dataatype to object type
df["Postcode"] = df["Postcode"].astype(str)
df["Postcode"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 27114 entries, 0 to 27113
Series name: Postcode
Non-Null Count  Dtype 
--------------  ----- 
27114 non-null  object
dtypes: object(1)
memory usage: 212.0+ KB


We have successfully changed the datatype of 'Postcode' variable to 'object'.

**Fixing the datatypes of 'Bedroom', 'Bathroom' and 'Car' variables from float to int**

In [48]:
# Converting the data types of 'Bedroom', 'Bathroom' and 'Car' to int
df["Bedroom"] = df["Bedroom"].astype("Int64")
df["Bathroom"] = df["Bathroom"].astype("Int64")
df["Car"] = df["Bedroom"].astype("Int64")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27114 entries, 0 to 27113
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         27114 non-null  object        
 1   Rooms          27114 non-null  int64         
 2   Type           27114 non-null  object        
 3   SellerG        27114 non-null  object        
 4   Date           27114 non-null  datetime64[ns]
 5   Distance       27113 non-null  float64       
 6   Postcode       27114 non-null  object        
 7   Bedroom        20678 non-null  Int64         
 8   Bathroom       20672 non-null  Int64         
 9   Car            20678 non-null  Int64         
 10  Landsize       17873 non-null  float64       
 11  BuildingArea   10543 non-null  object        
 12  YearBuilt      11985 non-null  float64       
 13  Regionname     27114 non-null  object        
 14  Propertycount  27114 non-null  int64         
 15  Price          2711

We have successfully fixed the datatypes of 'Bedroom', 'Bathroom' and 'Car' variables from float to int

**Fixing the datatype of 'YearBuilt' variable from float to int**

In [49]:
# Check why the YearBuilt variable reads as a float variable
df["YearBuilt"].unique()

array([2016., 1900.,   nan, 1930., 2013., 2015., 1965., 1970., 1964.,
       1950., 1980., 1920., 1985., 2000., 1960., 1974., 1997., 2008.,
       1940., 1905., 1957., 1890., 1935., 1989., 2009., 1955., 1975.,
       1910., 2004., 2010., 1880., 2007., 1925., 2001., 2005., 1904.,
       1995., 1990., 2012., 1993., 2003., 1978., 1998., 1889., 2014.,
       2017., 1977., 2006., 2011., 2002., 1919., 1915., 1969., 1863.,
       1893., 1912., 1968., 1972., 1999., 1943., 1951., 1922., 1996.,
       1992., 1958., 1962., 1967., 1934., 1887., 1976., 1945., 1973.,
       1994., 1983., 1982., 1918., 1986., 1953., 1928., 1948., 1941.,
       1949., 1988., 1991., 1924., 1897., 1923., 1984., 1987., 1856.,
       1902., 1966., 1961., 1929., 1952., 1885., 1938., 1870., 1971.,
       1903., 1906., 1868., 1907., 1963., 1939., 1927., 1956., 1979.,
       1860., 1884., 1926., 1888., 1954., 1894., 1937., 1981., 1959.,
       1914., 1895., 1921., 1913., 1936., 1875., 1917., 1881., 1908.,
       1862., 2018.,

There are no fractional years. This was probably an error in database management. As there are some missing year values, we will use Int64 type.

In [52]:
# converting the datatype of 'YearBuilt' to Int64
df["YearBuilt"] = df["YearBuilt"].astype("Int64")
df["YearBuilt"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 27114 entries, 0 to 27113
Series name: YearBuilt
Non-Null Count  Dtype
--------------  -----
11985 non-null  Int64
dtypes: Int64(1)
memory usage: 238.4 KB


We have successfully fixed the datatype of "YearBuilt" from float to int

**Fixing the datatype of 'BuildingArea' variable from object to float**

In [54]:
# Check why the variable reads as an object type
df["BuildingArea"].unique()

array(['225', '82', 'inf', '263', '242', '251', '117', 'missing', '76',
       '399', '118', '103', '180', '123', '218', '129', '167', '154',
       '275', '121', nan, '125', '255', '75', '156', '240', '268', '108',
       '69', '140', '214', '253', '189', '215', '96', '104', '100', '313',
       '144', '93', '110', '70', '122', '51', '147', '113', '83', '56',
       '137', '85', '64', '175', '3558', '170', '265', '353', '138', '19',
       '116', '87', '74', '320', '300', '210', '120', '86', '97', '200',
       '106', '14', '161', '128', '185', '146', '133', '115', '143',
       '150', '195', '236', '276', '188', '179', '249', '141', '34', '73',
       '107', '84', '81', '207', '50', '264', '312', '235', '221', '183',
       '132', '160', '186', '78', '105', '145', '62', '220', '315', '181',
       '61', '112', '420', '226', '266', '410', '449', '356', '477',
       '250', '95', '190', '284', '247', '213', '209', '119', '111',
       '130', '348', '166', '44', '176', '98', '159', '79'

Besides the appropriate BuildingArea values, there are 'inf', and 'missing' value that are causing the variable to read as an object. These values represent missing entries in general. We will first replace them with numpy.nan value for consistency and finally convert the datatype of the entire variable to float.

In [58]:
# Replacing 'inf' and 'missing' with np.nan
df["BuildingArea"] = df["BuildingArea"].replace(["inf","missing"], np.nan)
"inf" in df["BuildingArea"]

False

We have successfully replaced "inf" and "missing" with np.nan

In [60]:
# Converting "BuildingArea" datatype to float
df["BuildingArea"] = df["BuildingArea"].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27114 entries, 0 to 27113
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         27114 non-null  object        
 1   Rooms          27114 non-null  int64         
 2   Type           27114 non-null  object        
 3   SellerG        27114 non-null  object        
 4   Date           27114 non-null  datetime64[ns]
 5   Distance       27113 non-null  float64       
 6   Postcode       27114 non-null  object        
 7   Bedroom        20678 non-null  Int64         
 8   Bathroom       20672 non-null  Int64         
 9   Car            20678 non-null  Int64         
 10  Landsize       17873 non-null  float64       
 11  BuildingArea   10529 non-null  float64       
 12  YearBuilt      11985 non-null  Int64         
 13  Regionname     27114 non-null  object        
 14  Propertycount  27114 non-null  int64         
 15  Price          2711

BuildingArea now reads correctly as a float variable.

#### Checking for duplicate records and variables with missing values

In [63]:
# Check for duplicate records
df.duplicated().sum()

np.int64(14)

There are 14 redundant records. We will remove them as they can potentially distort our analysis.

In [69]:
# Removing duplicate records
df.drop_duplicates(inplace=True)
# Resetting the indexing
df.reset_index(drop=True, inplace=True)

#confirming removal of duplicates
df.duplicated().sum()

np.int64(0)

Duplicate records have been removed successfully.

**Which variables have missing values and to what proportion?**

In [74]:
# Checking variables with missing values
df.isnull().sum()[df.isnull().sum()>0]

Distance            1
Bedroom          6425
Bathroom         6431
Car              6425
Landsize         9227
BuildingArea    16571
YearBuilt       15115
dtype: int64

In [76]:
# What proportion of values is missing in each variable?
df.isnull().sum()[df.isnull().sum()>0]/df.shape[0]*100

Distance        0.004
Bedroom        23.708
Bathroom       23.731
Car            23.708
Landsize       34.048
BuildingArea   61.148
YearBuilt      55.775
dtype: float64

* There are 7 variables with missing values: Distance, Bedroom, Bathroom, Car, Landsize, BuildingArea and YearBuilt.
* The missingness in Distance is not significant; we can impute the missing value with either mean or median dependinging on the distribution of the variable.
* 61% of values in BuildingArea are missing. This is a very large proportion to impute; to avoid compromising the accuracy of the dataset that would arise from imputing 61% of the values, we will remove this variable. Plus, this variable is not critical to the analysis.
* The proportion of missingness in Bedroom, Bathroom, Car and Landsize can be imputed without jeopardizing the accuracy of the dataset.
* For the YearBuilt, the proportion of missingness is also significant(55%). Removing the variable would lead to loss of significant information to our analysis. Instead, we will create a copy of the dataset and drop the records with missing values for YearBuilt from this copy. In the original dataset, we will drop this variable.


#### Handling missing values

**Missing values in BuildingArea**

In [80]:
# Removing 'BuildingArea' variable from the dataset.
df.drop(columns=["BuildingArea"], inplace=True, errors="ignore")
df.head()

,Suburb,Rooms,Type,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,YearBuilt,Regionname,Propertycount,Price
0,Airport West,3,t,Nelson,2016-09-03,13.500,3042.0,3,2,3,303.000,2016,Western Metropolitan,3464,840000
1,Albert Park,2,h,hockingstuart,2016-09-03,3.300,3206.0,2,1,2,120.000,1900,Southern Metropolitan,3280,1275000
2,Albert Park,2,h,Thomson,2016-09-03,3.300,3206.0,2,1,2,159.000,<NA>,Southern Metropolitan,3280,1455000
3,Alphington,4,h,Brace,2016-09-03,6.400,3078.0,3,2,3,853.000,1930,Northern Metropolitan,2211,2000000
4,Alphington,3,h,Jellis,2016-09-03,6.400,3078.0,3,2,3,208.000,2013,Northern Metropolitan,2211,1110000
